In [1]:
import numpy as np
import pandas
import zipfile
import random
import pickle

In [2]:
with zipfile.ZipFile('full_dataset.csv.zip') as zipped:
    with zipped.open('full_dataset.csv') as file:
        df = pandas.read_csv(file)

# display dataset
# print(recipeNLG_df.head())

In [3]:
recipeNLG_df = df.loc[:, ['NER']]

In [4]:
# recipeNLG_df['NER'] = recipeNLG_df['NER'].apply(lambda x: x.replace(" ", "_").replace(",_", ", "))
# print(recipeNLG_df.head())

In [4]:
# https://towardsdatascience.com/dealing-with-list-values-in-pandas-dataframes-a177e534f173
recipeNLG_df['NER'] = recipeNLG_df.loc[:,'NER'].apply(eval)
print(recipeNLG_df.head())

                                                 NER
0  [brown sugar, milk, vanilla, nuts, butter, bit...
1  [beef, chicken breasts, cream of mushroom soup...
2  [frozen corn, cream cheese, butter, garlic pow...
3  [chicken, chicken gravy, cream of mushroom sou...
4  [peanut butter, graham cracker crumbs, butter,...


In [5]:
# load dataframe of node_ids and ingredients
id_ing_df = pandas.read_csv('nodes_191120.csv')

In [6]:
# open and load data from pickle file
file = open('FlavorGraph_NodeEmbedding.pickle', 'rb')

# keys are strings for node_id, value is embedding
data = pickle.load(file)

file.close()

# just get the embeddings for ingredients
ing_embeddings = {}

for i in range(len(id_ing_df)):
    if id_ing_df.loc[i, 'node_type'] == "ingredient":
        # map the name of the ingredient to the embedding
        ing_embeddings[id_ing_df.loc[i, 'name']] = data[str(id_ing_df.loc[i, 'node_id'])]
        

In [60]:
# print(ing_embeddings['brown_sugar'])

In [7]:
def clean_ing(ing_list):
    for i in range(len(ing_list)):
        ing_str = ing_list[i]
        ing_str = ing_str.lower()
        if ing_str.endswith('s'):
            ing_str = ing_str[:-1]    
        ing_str = ing_str.replace(" ", "_")
        ing_list[i] = ing_str
    return ing_list

def get_embeddings(ing_list):
    emb_list = []
    for i in range(len(ing_list)):
        ing_str = ing_list[i]

        emb = ing_embeddings.get(ing_str, [])
        if(len(emb) == 0):
            return []
        else:
            emb_list.append(emb)
    return emb_list

In [8]:
recipeNLG_df['NER'] = recipeNLG_df['NER'].apply(clean_ing)
recipeNLG_df['embeddings'] = recipeNLG_df['NER'].apply(get_embeddings)
print(recipeNLG_df.iloc[:10])

                                                 NER  \
0  [brown_sugar, milk, vanilla, nut, butter, bite...   
1  [beef, chicken_breast, cream_of_mushroom_soup,...   
2  [frozen_corn, cream_cheese, butter, garlic_pow...   
3  [chicken, chicken_gravy, cream_of_mushroom_sou...   
4  [peanut_butter, graham_cracker_crumb, butter, ...   
5  [baking_potatoe, extra_lean_ground_beef, butte...   
6  [sugar, butter, egg, buttermilk, flour, salt, ...   
7  [cream-style_corn, whole_kernel_corn, cracker,...   
8  [tomatoe, water, onion, worcestershire_sauce, ...   
9  [pineapple, condensed_milk, lemon, pecan, grah...   

                                          embeddings  
0                                                 []  
1  [[-0.33506337, 0.08803183, -0.24923237, 0.0341...  
2                                                 []  
3                                                 []  
4  [[-0.17900778, -0.039890748, -0.13081385, 0.30...  
5                                                 [] 

In [9]:
print(recipeNLG_df.head())
save = recipeNLG_df

# # load dataframe of node_ids and ingredients
# nodes_df = pandas.read_csv('nodes_191120.csv')
# ing_set = set(nodes_df['name'])
# print(ing_set)

                                                 NER  \
0  [brown_sugar, milk, vanilla, nut, butter, bite...   
1  [beef, chicken_breast, cream_of_mushroom_soup,...   
2  [frozen_corn, cream_cheese, butter, garlic_pow...   
3  [chicken, chicken_gravy, cream_of_mushroom_sou...   
4  [peanut_butter, graham_cracker_crumb, butter, ...   

                                          embeddings  
0                                                 []  
1  [[-0.33506337, 0.08803183, -0.24923237, 0.0341...  
2                                                 []  
3                                                 []  
4  [[-0.17900778, -0.039890748, -0.13081385, 0.30...  


In [10]:
# only keeps rows with NER list > 0
recipeNLG_df = recipeNLG_df[recipeNLG_df['embeddings'].map(lambda d: len(d)) > 0]

In [11]:
print(recipeNLG_df.head())

                                                  NER  \
1   [beef, chicken_breast, cream_of_mushroom_soup,...   
4   [peanut_butter, graham_cracker_crumb, butter, ...   
9   [pineapple, condensed_milk, lemon, pecan, grah...   
12                   [chicken, flour, barbecue_sauce]   
14  [pie_filling, pineapple, condensed_milk, lemon...   

                                           embeddings  
1   [[-0.33506337, 0.08803183, -0.24923237, 0.0341...  
4   [[-0.17900778, -0.039890748, -0.13081385, 0.30...  
9   [[0.16943195, -0.31285393, -0.23219007, 0.1775...  
12  [[-0.22679155, 0.100118645, 0.10348936, 0.2072...  
14  [[-0.021126581, 0.1452302, -0.09510492, -0.035...  


In [12]:
recipeNLG_df.loc[:, 'num_ingredients'] = recipeNLG_df.apply (lambda row: len(row.NER), axis=1)
recipeNLG_df.loc[:, 'start_ingredient'] = recipeNLG_df.apply (lambda row: random.choice(row.NER) if row.num_ingredients > 0 else '', axis=1)
print(recipeNLG_df.head())

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


                                                  NER  \
1   [beef, chicken_breast, cream_of_mushroom_soup,...   
4   [peanut_butter, graham_cracker_crumb, butter, ...   
9   [pineapple, condensed_milk, lemon, pecan, grah...   
12                   [chicken, flour, barbecue_sauce]   
14  [pie_filling, pineapple, condensed_milk, lemon...   

                                           embeddings  num_ingredients  \
1   [[-0.33506337, 0.08803183, -0.24923237, 0.0341...                4   
4   [[-0.17900778, -0.039890748, -0.13081385, 0.30...                5   
9   [[0.16943195, -0.31285393, -0.23219007, 0.1775...                5   
12  [[-0.22679155, 0.100118645, 0.10348936, 0.2072...                3   
14  [[-0.021126581, 0.1452302, -0.09510492, -0.035...                4   

          start_ingredient  
1   cream_of_mushroom_soup  
4           powdered_sugar  
9                    lemon  
12                   flour  
14               pineapple  


In [13]:
recipeNLG_df

,NER,embeddings,num_ingredients,start_ingredient
1,"[beef, chicken_breast, cream_of_mushroom_soup,...","[[-0.33506337, 0.08803183, -0.24923237, 0.0341...",4,cream_of_mushroom_soup
4,"[peanut_butter, graham_cracker_crumb, butter, ...","[[-0.17900778, -0.039890748, -0.13081385, 0.30...",5,powdered_sugar
9,"[pineapple, condensed_milk, lemon, pecan, grah...","[[0.16943195, -0.31285393, -0.23219007, 0.1775...",5,lemon
12,"[chicken, flour, barbecue_sauce]","[[-0.22679155, 0.100118645, 0.10348936, 0.2072...",3,flour
14,"[pie_filling, pineapple, condensed_milk, lemon...","[[-0.021126581, 0.1452302, -0.09510492, -0.035...",4,pineapple
...,...,...,...,...
2231129,"[brown_sugar, sour_cream, butter, butter, corn...","[[-0.37061608, -0.21077241, -0.011841524, -0.1...",14,sugar
2231132,"[egg, vegetable_oil, white_sugar, pumpkin_pure...","[[0.25692862, 0.08700464, 0.23288268, 0.169938...",15,salt
2231135,"[bread, italian_sauce, frozen_meatball]","[[0.17803018, 0.19869211, 0.01149651, 0.133690...",3,frozen_meatball
2231137,"[chocolate_hazelnut_spread, tortilla, butter, ...","[[0.16573386, -0.09754042, -0.16330065, 0.2495...",5,chocolate_hazelnut_spread


In [15]:
recipeNLG_df.to_pickle("ner_embeddings.pkl")

In [16]:
# open and load data from pickle file
check_pickle = open('ner_embeddings.pkl', 'rb')

# keys are strings for node_id, value is embedding
verify_data = pickle.load(check_pickle)

check_pickle.close()

In [17]:
verify_data.head()

,NER,embeddings,num_ingredients,start_ingredient
1,"[beef, chicken_breast, cream_of_mushroom_soup,...","[[-0.33506337, 0.08803183, -0.24923237, 0.0341...",4,cream_of_mushroom_soup
4,"[peanut_butter, graham_cracker_crumb, butter, ...","[[-0.17900778, -0.039890748, -0.13081385, 0.30...",5,powdered_sugar
9,"[pineapple, condensed_milk, lemon, pecan, grah...","[[0.16943195, -0.31285393, -0.23219007, 0.1775...",5,lemon
12,"[chicken, flour, barbecue_sauce]","[[-0.22679155, 0.100118645, 0.10348936, 0.2072...",3,flour
14,"[pie_filling, pineapple, condensed_milk, lemon...","[[-0.021126581, 0.1452302, -0.09510492, -0.035...",4,pineapple
